In [1]:
import pandas as panda

import numpy as np

import datetime, time

from matplotlib.pyplot import plot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns


from sklearn.linear_model import LinearRegression, RANSACRegressor, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [2]:
historical_transaction_data_path = 'all/historical_transactions.csv'

historical_transaction_data = panda.read_csv(historical_transaction_data_path)

historical_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [3]:
historical_transaction_data['card_id'] = historical_transaction_data['card_id'].apply(lambda x: x.upper().strip())

In [4]:
historical_transaction_data.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4E6213E9BC,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4E6213E9BC,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4E6213E9BC,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4E6213E9BC,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4E6213E9BC,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [5]:
historical_transaction_data.isnull().sum()

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               178159
merchant_category_id          0
merchant_id              138481
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2652864
state_id                      0
subsector_id                  0
dtype: int64

###### same treatment with category_3,and_2. they are categorical values, replace with highest occuring values.
###### remove rows with empty merchant:: qq: purchase amounts are available for these rows. we are not really considering merhant_id. then why are we removing them. we will not remove empty merchant id rows

In [6]:
historical_transaction_data['category_2'].fillna(historical_transaction_data.category_2.value_counts().idxmax(), inplace = True)

In [7]:
historical_transaction_data['category_3'].fillna(historical_transaction_data.category_3.value_counts().idxmax(), inplace = True)

In [9]:
historical_transaction_data.isnull().sum()

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
installments                 0
category_3                   0
merchant_category_id         0
merchant_id             138481
month_lag                    0
purchase_amount              0
purchase_date                0
category_2                   0
state_id                     0
subsector_id                 0
dtype: int64

In [18]:
total_transaction = historical_transaction_data.groupby(['card_id'])['city_id'].count()

total_transaction = total_transaction.reset_index()

total_transaction.columns = ['card_id','no_of_transaction']

total_transaction.head()

,card_id,no_of_transaction
0,C_ID_00007093C1,149
1,C_ID_0001238066,123
2,C_ID_0001506EF0,66
3,C_ID_0001793786,216
4,C_ID_000183FDDA,144


In [ ]:
total_transaction_merged = historical_transaction_data.merge(total_transaction, how='right', on ='card_id')

total_transaction_merged.head()

In [14]:
len(total_transaction_merged[total_transaction_merged['card_id']=='C_ID_4E6213E9BC'])

400

In [19]:
total_transaction[total_transaction['card_id']=='C_ID_4E6213E9BC']

,card_id,no_of_transaction
99809,C_ID_4E6213E9BC,400
